In [1]:
# 科技股轮动 macd 

import backtrader as bt
import backtrader.indicators as bi
import backtest

import pandas as pd
import math
import matplotlib.pyplot as plt
import datetime
import os
from futu import OpenQuoteContext
import talib as ta
from numpy import array

import sys   
sys.setrecursionlimit(100000)

In [2]:
class Tech_roll(bt.Strategy):
    params =dict(
            myperiod = 120,
            printlog = False,
            setroll = 1,
            setboll = 0)
    
    
    def __init__(self):
        self.order = None
        self.buyprice = 0
        self.comm = 0
        self.buy_size = 0
        self.buy_count = 0
        self.printlog = True

        # 自定义的指标
        self.df_stock=df_stock
        #self.daily_volume=df_cn
        self.stocklist = code
        self.boll_signaltag = None
        self.HSI_data =df_HSI
        

    def get_daily_list(self):

        MA=13
        i=0 # 计数器初始化
        # 创建保持计算结果的DataFrame
        df = pd.DataFrame()
        date = self.datas[0].datetime.date(0).isoformat()
        for security in self.stocklist:
            # 获取股票的收盘价            
            close_data = self.df_stock[security]
            # 获取股票现价
            idx= close_data.index.get_loc(date)
            if idx < MA:                
                return df
            close_data = close_data.iloc[idx - MA+1 :idx+1]['close']            
            current_price = close_data.iloc[-1]                                   
            cp_increase = (current_price/close_data[0]-1)*100            
            # 取得平均价格
            ma_n1 = close_data.mean()
            # 计算前一收盘价与均值差值    
            pre_price = (current_price/ma_n1-1)*100            
            df.loc[i,'股票代码'] = security # 把标的股票代码添加到DataFrame
            df.loc[i,'股票名称'] = 'tx' # 把标的股票名称添加到DataFrame
            df.loc[i,'周期涨幅'] = cp_increase # 把计算结果添加到DataFrame
            df.loc[i,'均线差值'] = pre_price # 把计算结果添加到DataFrame
            i=i+1                     
        df = df.fillna(-100)
        df.sort_values(by='周期涨幅',ascending=False,inplace=True) # 按照涨幅排序        
        df.reset_index(drop=True, inplace=True) # 重新设置索   
        for t in df.index:
            if df.loc[t,'周期涨幅'] < 0.1 or df.loc[t,'均线差值'] < 0:
                df=df.drop(t)
        if self.p.setroll == 0:
            return self.stocklist
        else:
            return df['股票代码'].tolist()[0:2]


    def get_boll(self,date):
        """
        获取北向资金布林带    
        
        净买入额在布林线下轨以下时清仓
        净买入额在布林线上轨以上时执行调仓
        """        
        if self.p.setboll ==0:
            return 'up'
        
        stdev_n = 2
        money_df = self.daily_volume
               
        if len(money_df[money_df.day==date]) <1 :
                        
            return 'None'
        
        idx=money_df[money_df['day']==date].index
        idx=idx[-1]
        money_df=money_df.loc[:idx].copy()
        money_df['net_amount'] = money_df['quota_daily'] - money_df['quota_daily_balance'] #每日额度-每日剩余额度=净买入额
        # 分组求和
        money_df = money_df.groupby('day')[['net_amount']].sum().iloc[-self.p.myperiod:] #过去self.params.myperio天求和
        mid = money_df['net_amount'].mean()
        stdev = money_df['net_amount'].std()
        upper = mid + stdev_n * stdev
        lower = mid - stdev_n * stdev
        mf = money_df['net_amount'].iloc[-1]
        
        if mf >=int(upper):
            self.boll_signaltag ='UPCROSS'
            return 'up'
                        
        # 净买入额在布林线下轨以下时清仓
        elif  mf <= lower:
            self.boll_signaltag ='DOWNCROSS'
            return 'down'
        else:
            return 'no signal'

    def get_macdSignal(self):
        date = self.datas[0].datetime.date(0).isoformat()
        HSI_data = self.HSI_data
        idx = HSI_data.index.get_loc(date)
        print(idx)
        pdata=self.HSI_data[idx-60+1:idx+1]
        idxc=pdata['close']
        price = array(idxc)
        macd = ta.MACD(price, fastperiod=5, slowperiod=15, signalperiod=7)[-1]

        if sum(macd[-20:-5])>0:
            if macd[-1]>0:
                return 0
            elif sum(macd[-5:])>0:
                return 1
        return 2
        
        


    def next(self):
        if self.order:
            return
        date = self.datas[0].datetime.date(0).isoformat()
        signal = self.get_macdSignal()
        print (date, ' ------',signal)
        
        
        if signal == 1:
            print('1 sell-----')
            for stock in self.datas: 
                print(stock._name)
                self.close(stock)
        if signal != 0:

            print( ' keep-----')
            return

        
        cash=self.broker.get_value()/len(self.stocklist)
            
        for stock in self.datas:
                            
            pos = self.getposition(stock)
            print ('len',len(pos))
            if len(pos)>0:
                print( 'keep  in pos')
                return
            else:    
                self.order_target_value(stock,cash*0.9)


        



    # 输出交易记录
    def log(self, txt, dt = None, doprint = False):
        if self.printlog :
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
            
    def notify_order(self, order):
        # 有交易提交/被接受，啥也不做
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 交易完成，报告结果
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    '执行买入, 价格: %.2f, 成本: %.2f, 手续费 %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.comm += order.executed.comm
            else:
                self.log(
                    '执行卖出, 价格: %.2f, 成本: %.2f, 手续费 %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.comm += order.executed.comm
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("交易失败")
        self.order = None
        
    # 输出手续费
    def stop(self):
        self.log("手续费:%.2f 成本比例:%.5f" % (self.comm, self.comm/self.broker.getvalue()))
    

In [4]:
def get_stockdata(stock_list,start_date,end_date):
    df={}
    for stock in stock_list:
        code = stock
        filename = code+".csv"
        path = "./stock/"        
        if not os.path.exists(path):
            os.makedirs(path)
        if os.path.exists(path + filename):
            df_stock = pd.read_csv(path + filename)    
        else:
            quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
            ret, df_stock, page_req_key = quote_ctx.request_history_kline(stock, start=start_date, end=end_date)  
            quote_ctx.close() 
            df_stock['date']=pd.to_datetime(df_stock['time_key'])        
            df_stock.to_csv(path + filename) 
        df_stock.index = pd.to_datetime(df_stock.date)
        df_stock['openinterest']=0
        df_stock = df_stock[['open','high','low','close','volume','openinterest']]
        df.update({stock:df_stock})

    return df

In [35]:
df_stock = get_stockdata(code,start,end)
backtest = backtest.BackTest(Tech_roll, start, end, code, name,500000,bDraw = True)
    
    
result = backtest.run()
#result = backtest.run()
#result = backtest.optRun(myperiod = range(50,150,20))
print(result)

AttributeError: 'BackTest' object has no attribute 'BackTest'

In [40]:
df_HSI = pd.read_csv('HSI.csv')

In [42]:
df_HSI['date']=pd.to_datetime(df_HSI['time_key']) 

In [45]:
df_HSI=df_HSI.set_index('date',drop=True)

In [46]:
df_HSI

,Unnamed: 0,code,time_key,open,close,high,low,pe_ratio,turnover_rate,volume,turnover,change_rate,last_close
date,,,,,,,,,,,,,
2016-02-19,0,HK.800000,2016-02-19 00:00:00,19238.43,19285.50,19342.03,19210.65,0.0,0.0,0,5.538735e+10,-0.400659,19363.08
2016-02-22,1,HK.800000,2016-02-22 00:00:00,19374.24,19464.09,19524.97,19374.24,0.0,0.0,0,6.019724e+10,0.926033,19285.50
2016-02-23,2,HK.800000,2016-02-23 00:00:00,19506.94,19414.78,19519.85,19328.26,0.0,0.0,0,5.873686e+10,-0.253338,19464.09
2016-02-24,3,HK.800000,2016-02-24 00:00:00,19282.34,19192.45,19360.80,19060.16,0.0,0.0,0,6.069822e+10,-1.145158,19414.78
2016-02-25,4,HK.800000,2016-02-25 00:00:00,19215.49,18888.75,19215.49,18867.70,0.0,0.0,0,6.368460e+10,-1.582393,19192.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-25,135,HK.800000,2020-09-25 00:00:00,23425.75,23235.42,23495.26,23124.25,0.0,0.0,0,1.167331e+11,-0.324524,23311.07
2020-09-28,136,HK.800000,2020-09-28 00:00:00,23284.09,23476.05,23523.81,23250.34,0.0,0.0,0,9.334340e+10,1.035617,23235.42
2020-09-29,137,HK.800000,2020-09-29 00:00:00,23584.61,23275.53,23601.40,23256.92,0.0,0.0,0,8.588219e+10,-0.854147,23476.05
